In [11]:
import pandas as pd
from shot_influence import plot_shot_influence
from Utils.Dataprocess import *
File_list = ['../input_data/all_dataset.csv']

def Evaluation(file):
    """所對應分別是：
    df: dataframe
    df_dict: df長度統計dict
    df_List: df根據rally切分dataframe的list
    df_len: df的rally數量
    """

    df, df_dict, df_List, df_len =  ReadFile(file)
    player_list = df['player'].unique()
    print('Players:', player_list)

    if len(player_list) > 2:
        print('Too much player')
        return

    fileModel1 = f"../Shot_Evaluation/win_prob_data/dataset_{player_list[0]}.csv"
    df_model = pd.read_csv(fileModel1)

    match_id = df.iloc[0]["match_id"]
    set_num = df.iloc[0]["set"]
    for i in range(df_len):
        plot_shot_influence.main(player_list[0], match_id, set_num, i+1, df, df_model)

    return player_list


def main(File_list):
    for file in File_list:
        player_list = Evaluation(file)

main(File_list)


Players: ['Kento MOMOTA' 'CHOU Tien Chen']


KeyError: 'rally_id'

In [1]:
import numpy as np
import pandas as pd
import os
from Utils.coord2area import coord2area
from Utils.Dataprocess import *
from Utils.PlotLastBallRound import plot_last_ball_round
from Utils.PlotShotTypeEvaluation import plot_shot_type_evaluation
from Utils.PlotTopReasons import plot_top_reasons
from shot_influence import plot_shot_influence


def Evaluation(match, last_ball_round = 3, dest_folder = '../Shot_Evaluation/Result'):
    """所對應分別是：
    df: dataframe
    df_dict: df長度統計dict
    df_List: df根據rally切分dataframe的list
    df_len: df的rally數量
    """

    df, df_dict, df_List, df_len = ReadDataFrame(match)
    player_list = df["player"].unique()
    print("Players:", player_list)

    if len(player_list) > 2:
        print("Too much player")
        return
    
    os.makedirs(dest_folder, exist_ok=True)

    # player_list正常來說會有兩個人，因此可以兩個人都分析
    fileModel1 = f"../Shot_Evaluation/win_prob_data/dataset_{player_list[0]}.csv"
    fileModel2 = f"../Shot_Evaluation/win_prob_data/dataset_{player_list[1]}.csv"
    df_model1 = pd.read_csv(fileModel1)
    df_model2 = pd.read_csv(fileModel2)

    # 1. 選手的位置＆球的落點分佈
    fn1 = plot_shot_type_evaluation(df, player_list[0], dest_folder)
    fn2 = plot_shot_type_evaluation(df, player_list[1], dest_folder)
    print("Plot location distribution")


    # 2. 最後n球 選手的位置＆球的落點分佈
    fn3 = plot_last_ball_round(df, player_list[0], last_ball_round, dest_folder)
    fn4 = plot_last_ball_round(df, player_list[1], last_ball_round, dest_folder)
    print("Plot last 3 ball rounds distribution")


    # 3. 贏球輸球的原因
    fn5 = plot_top_reasons(df_List, player_list[0], dest_folder)
    fn6 = plot_top_reasons(df_List, player_list[1], dest_folder)
    print("Top 3 reasons for winning/losing")
    
    # 4. Shot influence
    fn7 = []
    fn8 = []
    match_id = df.iloc[0]["match_id"]
    set_num = df.iloc[0]["set"]
    rallies = df[(df["match_id"] == match_id)
        & (df["set"] == set_num)]["rally"].unique()
    for i in rallies:
        fn7.append(plot_shot_influence.main(player_list[0], match_id, set_num, i, df, df_model1, dest_folder))
    for i in rallies:
        fn8.append(plot_shot_influence.main(player_list[1], match_id, set_num, i, df, df_model2, dest_folder))
    #fn7 = plot_shot_influence.main(player_list[0], 23, 1, 17, df, df_model1, dest_folder)
    #fn8 = plot_shot_influence.main(player_list[1], 23, 1, 17, df, df_model2, dest_folder)

    return {player_list[0]: {'shot_type': fn1, 'last_ball': fn3, 'top_reasons': fn5, 'shot_influence': fn7},
            player_list[1]: {'shot_type': fn2, 'last_ball': fn4, 'top_reasons': fn6, 'shot_influence': fn8}}


def main(File_list):
    """
    格式:File = [file 1, file 2,...file n]
    file n = {player 1: {plot1_1: path1_1, plot1_2: path1_2, ...}, 
              player 2: {plot2_1: path2_1, plot2_2: path2_2, ...}}
    """
    File = []
    for file in File_list:
        match = pd.read_csv(file)
        file_dict = Evaluation(match)
        File.append(file_dict)

    return File


if __name__ == "__main__":
    File_list = ["../input_data/all_dataset.csv"]
    main(File_list)


2024-09-29 23:14:33.609787: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-29 23:14:33.611011: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-29 23:14:33.632698: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-29 23:14:33.633233: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-29 23:14:33.972607: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

Players: ['Kento MOMOTA' 'CHOU Tien Chen']


/home/derrick/Downloads/from_gdwang-New_Environment/Shot_Evaluation/Utils/PlotShotTypeEvaluation.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Real.loc[:, "scaled_landing_x"] = df_Real["landing_x"] * 177.5 + 177.5
/home/derrick/Downloads/from_gdwang-New_Environment/Shot_Evaluation/Utils/PlotShotTypeEvaluation.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Real.loc[:, "scaled_landing_y"] = 240 - df_Real["landing_y"] * 240
/home/derrick/Downloads/from_gdwang-New_Environment/Shot_Evaluation/

Plot location distribution
Plot last 3 ball rounds distribution


/home/derrick/Downloads/from_gdwang-New_Environment/Shot_Evaluation/shot_influence/plot_shot_influence.py:451: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values


Top 3 reasons for winning/losing


posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and pos

In [7]:
#shot type evaluation
import os
import uuid

import matplotlib.pyplot as plt

from Utils.AnalyzeShotsbyGrid import (
    analyze_shots_by_grid_singles,
    find_extremes_in_grid_singles,
)
from Utils.DrawCourt import draw_half_court
from Utils.GenerateGridDescriptions import generate_grid_descriptions_singles


def plot_shot_type_evaluation(
    df_Real, dest_folder="./Shot_Evaluation/Result"
):
    os.makedirs(dest_folder, exist_ok=True)

    # Scale the landing positions
    df_Real.loc[:, "scaled_landing_x"] = df_Real["landing_x"] * 177.5 + 177.5
    df_Real.loc[:, "scaled_landing_y"] = 240 - df_Real["landing_y"] * 240

    # Get unique shot types
    shot_types = df_Real["type"].unique()
    excluded_types = ["Serve short", "Serve long"]
    shot_types = [
        shot_type for shot_type in shot_types if shot_type not in excluded_types
    ]

    # Create subplots
    fig, axs = plt.subplots(1, len(shot_types), figsize=(20, 5))
    axs = axs.flatten()

    shot_stats = []
    # Plot the distributions for each shot type
    for i, shot_type in enumerate(shot_types):
        shot_df = df_Real[df_Real["type"] == shot_type]
        total_shots = len(shot_df)
        out_df = shot_df[shot_df["lose_reason"] == "Out of bound"]
        total_outs = len(out_df)
        net_df = shot_df[shot_df["lose_reason"] == "Net"]
        total_nets = len(net_df)
        win_df = shot_df[shot_df["lose_reason"] == "Miss"]
        total_wins = len(win_df)

        error_rate = (total_outs + total_nets) / total_shots if total_shots > 0 else 0
        win_rate = total_wins / total_shots if total_shots > 0 else 0
        shot_stats.append(
            {
                "shot_type": shot_type,
                "total_shots": total_shots,
                "win_rate": win_rate,
                "error_rate": error_rate,
                "total_wins": total_wins,
                "total_errors": total_outs + total_nets,
            }
        )

        # Draw the court on the subplot
        draw_half_court(ax=axs[i])

        # Plot individual landing points
        axs[i].scatter(
            out_df["scaled_landing_x"],
            out_df["scaled_landing_y"],
            c="green",
            label="Out",
            alpha=0.5,
        )
        axs[i].scatter(
            net_df["scaled_landing_x"],
            net_df["scaled_landing_y"],
            c="red",
            label="Net",
            alpha=0.5,
        )
        axs[i].scatter(
            win_df["scaled_landing_x"],
            win_df["scaled_landing_y"],
            c="blue",
            label="Win",
            alpha=0.5,
        )

        # Add title and legend
        axs[i].set_title(
            f"{shot_type}\nError Rate: {total_outs + total_nets}/{total_shots}({error_rate:.2%})\nWin Rate: {total_wins}/{total_shots}({win_rate:.2%})"
        )
        axs[i].legend(loc="upper right")

    fig.suptitle(f"Shot Type Evaluation - All Players", fontsize=16, y=0.98)

    # Save the plot
    plt.tight_layout()
    filename = f"{dest_folder}/shot_type_evaluation.png"
    plt.savefig(filename)
    plt.close()

    # Save the descriptions as text files
    """real_txt_filename = f'./Shot_Evaluation/Result/real_matches_description_{player_name}.txt'
    grid_txt_filename = f'./Shot_Evaluation/Result/grid_description_singles_{player_name}.txt'

    # Save real matches description
    with open(real_txt_filename, 'w') as f:
        f.write("\n".join(real_matches_description))

    # Save grid description
    with open(grid_txt_filename, 'w') as f:
        f.write(grid_description_singles)"""


df = pd.read_csv("../input_data/All_dataset.csv")
type = {'發短球': 'Serve short', '長球': 'Clear', '推撲球': 'Push Shot', 
            '殺球': 'Smash', '接殺防守': 'Smash Defence', '平球': 'Drive', 
            '接不到': 'Missed shot', '網前球': 'Net Shot', '挑球': 'Lob', 
            '切球': 'Drop', '發長球': 'Serve long', '點扣': 'Wrist smash',
            '擋小球': 'Return Net', '放小球': 'Net shot'}
    
lose_reason = {'出界': 'Out of bound', '未過網': 'Net', '對手落地致勝': 'Miss', '掛網': 'Net', '落點判斷失誤': 'Miss'}
    
    
df['type'] = df['type'].map(type)
df['lose_reason'] = df['lose_reason'].map(lose_reason)
df = df[df['type'] != 'Missed shot'].reset_index(drop=True)

dest_folder = '../Shot_Evaluation/Result/All_players'
plot_shot_type_evaluation(df, dest_folder)


In [8]:
#last ball round
from Utils.CoordinatesProcess import transform_coordinates
import pandas as pd
import os
import matplotlib.pyplot as plt
from Utils.DrawCourt import draw_full_court
import seaborn as sns
import matplotlib.patches as mpatches
from Utils.GenerateShotandMovementDescription import *
import uuid

def plot_last_ball_round(df, last_n, dest_folder = './Shot_Evaluation/Result'):
    os.makedirs(dest_folder, exist_ok=True)
    
    df['scaled_landing_x'] = df['landing_x'] * 177.5 + 177.5
    df['scaled_landing_y'] = 240 - df['landing_y'] * 240 + 480
    df['scaled_moving_x'] = 177.5 - df['moving_x'] * 177.5
    df['scaled_moving_y'] = df['moving_y'] * 240 + 240
    
    last_df = df.groupby(['match_id', 'set', 'rally']).tail(last_n)

    # Get unique shot types
    shot_types = df['type'].unique()
    excluded_types = ["Serve short", "Serve long"]
    shot_types = [shot_type for shot_type in shot_types if shot_type not in excluded_types]

    # Create subplots
    fig, axs = plt.subplots(1, len(shot_types), figsize=(20, 2 * 6))
    axs = axs.flatten()

    # Plot the distributions for each shot type
    for i, shot_type in enumerate(shot_types):

        # Filter for specific shot types
        shot_df = last_df[last_df['type'] == shot_type]

        # Draw the court on the subplot
        draw_full_court(ax=axs[i])

        # Plot landing points using KDE (Kernel Density Estimate)
        sns.kdeplot(x=shot_df['scaled_landing_x'], y=shot_df['scaled_landing_y'], ax=axs[i], cmap='Greens', fill=True, alpha=0.5)
        
        # Plot movement after shots
        sns.kdeplot(x=shot_df['scaled_moving_x'], y=shot_df['scaled_moving_y'], ax=axs[i], cmap='Greens', fill=True, alpha=0.5)

        # Add title
        axs[i].set_title(f'{shot_type}')

    # Set the title for the entire figure
    fig.suptitle(f'Last {last_n} Ball Rounds Distribution - All Players', fontsize=16, y=0.98)

    # Adjust layout
    plt.tight_layout()

    # Save the plot
    filename = f'{dest_folder}/last_ball_round.png'
    plt.savefig(filename)
    plt.close()
    

df = pd.read_csv("../input_data/All_dataset.csv")
type = {'發短球': 'Serve short', '長球': 'Clear', '推撲球': 'Push Shot', 
            '殺球': 'Smash', '接殺防守': 'Smash Defence', '平球': 'Drive', 
            '接不到': 'Missed shot', '網前球': 'Net Shot', '挑球': 'Lob', 
            '切球': 'Drop', '發長球': 'Serve long', '點扣': 'Wrist smash',
            '擋小球': 'Return Net', '放小球': 'Net shot'}
    
lose_reason = {'出界': 'Out of bound', '未過網': 'Net', '對手落地致勝': 'Miss', '掛網': 'Net', '落點判斷失誤': 'Miss'}
    
    
df['type'] = df['type'].map(type)
df['lose_reason'] = df['lose_reason'].map(lose_reason)
df = df[df['type'] != 'Missed shot'].reset_index(drop=True)

dest_folder = '../Shot_Evaluation/Result/All_players'

plot_last_ball_round(df, 3, dest_folder)

In [ ]:
# top reasons
import os
import heapq
import pandas as pd
import matplotlib.pyplot as plt
from Utils.DrawCourt import draw_full_court
import uuid

def coordContinuous2Discrete(x, y, position):
    if position == 1:
        pass
    elif position == 2:
        x = -x
        y = -y
    else:
        raise NotImplementedError

    if y < 0:
        return 10
    elif y < 110:
        region = [1, 2, 3]
    elif y < 220:
        region = [4, 5, 6]
    elif y < 330:
        region = [7, 8, 9]
    else:
        return 10

    if x < -127.5:
        return 10
    elif x < -42.5:
        return region[2]
    elif x < 42.5:
        return region[1]
    elif x < 127.5:
        return region[0]
    else:
        return 10

def get_row_state(dataset, player_name, result):
    if len(dataset) < 2:
        return None, None
    if result == 'lose':
        row = dataset.iloc[-1] if dataset.iloc[-1]['player'] != player_name else dataset.iloc[-2]
    else:
        row = dataset.iloc[-1] if dataset.iloc[-1]['player'] == player_name else dataset.iloc[-2]
    player_region = coordContinuous2Discrete(row['opponent_location_x'] * 177.5, 240 - row['opponent_location_y'] * 240, 1)
    opponent_region = coordContinuous2Discrete(row['player_location_x'] * 177.5, 240 - row['player_location_y'] * 240, 1)
    ball_region = coordContinuous2Discrete(row['landing_x'] * 177.5, 240 - row['landing_y'] * 240, 1)
    state = (player_region, opponent_region, ball_region)
    lose_reason = dataset.iloc[-1]['lose_reason']

    return (state, lose_reason)

def plot_top_reasons(Real_List, player_name, dest_folder = './Shot_Evaluation/Result'):
    os.makedirs(dest_folder, exist_ok=True)
    
    win_reason_count_statistic = {}
    lose_reason_count_statistic = {}
    
    for df in Real_List:
        if df.iloc[-1]['getpoint_player'] == 'player':
            state, lose_reason = get_row_state(df, player_name, 'lose')
            if (state, lose_reason) not in lose_reason_count_statistic:
                lose_reason_count_statistic[(state, lose_reason)] = 0
            lose_reason_count_statistic[(state, lose_reason)] += 1
        else:
            state, lose_reason = get_row_state(df, player_name, 'win')
            if (state, lose_reason) not in win_reason_count_statistic:
                win_reason_count_statistic[(state, lose_reason)] = 0
            win_reason_count_statistic[(state, lose_reason)] += 1

    win_top4 = heapq.nlargest(4, win_reason_count_statistic.items(), key=lambda x: x[1])
    lose_top4 = heapq.nlargest(4, lose_reason_count_statistic.items(), key=lambda x: x[1])
    win_total = sum(win_reason_count_statistic.values())
    lose_total = sum(lose_reason_count_statistic.values())

    win_shots_dict = {key: [] for key, _ in win_top4}
    lose_shots_dict = {key: [] for key, _ in lose_top4}

    for df in Real_List:
        state, lose_reason = get_row_state(df, player_name, 'lose')
        if (state, lose_reason) in lose_shots_dict:
            shot = df.iloc[-1] if df.iloc[-1]['player'] != player_name else df.iloc[-2]
            lose_shots_dict[(state, lose_reason)].append(shot)
        state, lose_reason = get_row_state(df, player_name, 'win')
        if (state, lose_reason) in win_shots_dict:
            shot = df.iloc[-1] if df.iloc[-1]['player'] == player_name else df.iloc[-2]
            win_shots_dict[(state, lose_reason)].append(shot)

    win_shots_dfs = {key: pd.DataFrame(shots) for key, shots in win_shots_dict.items()}
    lose_shots_dfs = {key: pd.DataFrame(shots) for key, shots in lose_shots_dict.items()}

    # Plotting top 4 win shots
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))
    for index, (key, df) in enumerate(win_shots_dfs.items()):
        df['scaled_landing_x'] = df['landing_x'] * 177.5 + 177.5
        df['scaled_landing_y'] = 720 - df['landing_y'] * 240
        df['scaled_opponent_location_x'] = df['opponent_location_x'] * 177.5 + 177.5
        df['scaled_opponent_location_y'] = 720 - df['opponent_location_y'] * 240
        df['scaled_player_location_x'] = 177.5 - df['player_location_x'] * 177.5
        df['scaled_player_location_y'] = 240 + df['player_location_y'] * 240

        draw_full_court(ax=axs[index])

        axs[index].scatter(df['scaled_landing_x'], df['scaled_landing_y'], c='blue', label='Ball', alpha=0.5)
        axs[index].scatter(df['scaled_opponent_location_x'], df['scaled_opponent_location_y'], c='red', label='Opponent', alpha=0.5)
        axs[index].scatter(df['scaled_player_location_x'], df['scaled_player_location_y'], c='green', label='Player', alpha=0.5)

        axs[index].set_title(f'Player: {key[0][0]} Opponent: {key[0][1]} Ball: {key[0][2]}\nWin reason: {key[1]} Percent: {"{:.1%}".format(len(df)/win_total)}', fontsize=10)
        axs[index].legend(loc='upper right', fontsize=8)

    fig.suptitle(f'Top 4 Win Reasons - {player_name}', fontsize=16, y=0.98)
    plt.tight_layout()
    win_id = uuid.uuid4()
    filename = f'{dest_folder}/{win_id}.png'
    plt.savefig(filename)
    plt.close()

    # Plotting top 4 lose shots
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))
    for index, (key, df) in enumerate(lose_shots_dfs.items()):
        df['scaled_landing_x'] = df['landing_x'] * 177.5 + 177.5
        df['scaled_landing_y'] = 720 - df['landing_y'] * 240
        df['scaled_opponent_location_x'] = df['opponent_location_x'] * 177.5 + 177.5
        df['scaled_opponent_location_y'] = 720 - df['opponent_location_y'] * 240
        df['scaled_player_location_x'] = 177.5 - df['player_location_x'] * 177.5
        df['scaled_player_location_y'] = 240 + df['player_location_y'] * 240

        draw_full_court(ax=axs[index])

        axs[index].scatter(df['scaled_landing_x'], df['scaled_landing_y'], c='blue', label='Ball', alpha=0.5)
        axs[index].scatter(df['scaled_opponent_location_x'], df['scaled_opponent_location_y'], c='red', label='Opponent', alpha=0.5)
        axs[index].scatter(df['scaled_player_location_x'], df['scaled_player_location_y'], c='green', label='Player', alpha=0.5)

        axs[index].set_title(f'Player: {key[0][0]} Opponent: {key[0][1]} Ball: {key[0][2]}\nLose reason: {key[1]} Percent: {"{:.1%}".format(len(df)/lose_total)}', fontsize=10)
        axs[index].legend(loc='upper right', fontsize=8)

    fig.suptitle(f'Top 4 Lose Reasons - {player_name}', fontsize=16, y=0.98)
    loose_id = uuid.uuid4()
    filename = f'{dest_folder}/{loose_id}.png'
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

    return {"win": win_id, "loose": loose_id}

df = pd.read_csv("../input_data/All_dataset.csv")
type = {'發短球': 'Serve short', '長球': 'Clear', '推撲球': 'Push Shot', 
            '殺球': 'Smash', '接殺防守': 'Smash Defence', '平球': 'Drive', 
            '接不到': 'Missed shot', '網前球': 'Net Shot', '挑球': 'Lob', 
            '切球': 'Drop', '發長球': 'Serve long', '點扣': 'Wrist smash',
            '擋小球': 'Return Net', '放小球': 'Net shot'}
    
lose_reason = {'出界': 'Out of bound', '未過網': 'Net', '對手落地致勝': 'Miss', '掛網': 'Net', '落點判斷失誤': 'Miss'}
    
    
df['type'] = df['type'].map(type)
df['lose_reason'] = df['lose_reason'].map(lose_reason)
df = df[df['type'] != 'Missed shot'].reset_index(drop=True)

dest_folder = '../Shot_Evaluation/Result/All_players'

plot_last_ball_round(df, 3, dest_folder)

In [21]:
import numpy as np
import pandas as pd
import os
from Utils.coord2area import coord2area
from Utils.Dataprocess import *
from Utils.PlotLastBallRound import plot_last_ball_round
from Utils.PlotShotTypeEvaluation import plot_shot_type_evaluation
from Utils.PlotTopReasons import plot_top_reasons
from shot_influence import plot_shot_influence


def Evaluation(match, dest_folder = '../Shot_Evaluation/Result'):
    """所對應分別是：
    df: dataframe
    df_dict: df長度統計dict
    df_List: df根據rally切分dataframe的list
    df_len: df的rally數量
    """

    df, df_dict, df_List, df_len = ReadDataFrame(match)
    player_list = df["player"].unique()
    print("Players:", player_list)

    if len(player_list) > 2:
        print("Too much player")
        return
    
    os.makedirs(dest_folder, exist_ok=True)

    # player_list正常來說會有兩個人，因此可以兩個人都分析
    fileModel1 = f"../Shot_Evaluation/win_prob_data/dataset_{player_list[0]}.csv"
    fileModel2 = f"../Shot_Evaluation/win_prob_data/dataset_{player_list[1]}.csv"
    df_model1 = pd.read_csv(fileModel1)
    df_model2 = pd.read_csv(fileModel2)
    
    # 4. Shot influence
    match_id = df.iloc[0]["match_id"]
    set_num = df.iloc[0]["set"]
    rallies = df[(df["match_id"] == match_id)
        & (df["set"] == set_num)]["rally"].unique()
    for i in rallies:
        plot_shot_influence.main(player_list[0], match_id, set_num, i, df, df_model1, dest_folder)
    for i in rallies:
        plot_shot_influence.main(player_list[1], match_id, set_num, i, df, df_model2, dest_folder)



def main(File):
    """
    格式:File = [file 1, file 2,...file n]
    file n = {player 1: {plot1_1: path1_1, plot1_2: path1_2, ...}, 
              player 2: {plot2_1: path2_1, plot2_2: path2_2, ...}}
    """
    df = pd.read_csv(File)
    df['match_id'] = df['match_id'].astype(int).astype(str)
    match_id_mapping = {'1':'23', '3': '28', '5': '30', '6': '31', '7': '32', '13': '49', '2': '25', '4': '29', '8': '36', '9': '43', '10': '44',
                    '11': '45', '19': '55', '14': '50', '15': '51', '30': '72', '36': '79', '44': '97', '17': '53', '26': '64', '12': '48',
                    '20': '56', '23': '60', '28': '69', '41': '88', '18': '54', '16': '52', '27': '66', '33': '75', '37': '82', '24': '61',
                    '39': '86', '21': '57', '34': '76', '29': '71', '35': '78', '38': '85', '22': '58', '32': '74', '25': '63', '31': '73',
                    '43': '94', '42': '89', '40': '87'}
    match_id_mapping = {k: int(v) for k, v in match_id_mapping.items()}
    df['match_id'] = df['match_id'].map(match_id_mapping)
    match_ids = df["match_id"].unique()
    for match_id in match_ids:
        if match_id in [23,28,30,31,32,49,25,29,36,43,44,45,55,50,51,72,79,97,53,64,48,56,60,69,88,54,52,66,75,82,61,86,57,76,71,78,85,58,74,63,73,94,89,87]:
            #36, 72, 79, 88, 75, 82, 61, 86, 71, 58, 74, 63, 73, 94, 89, 87
            continue
        match_df = df[df["match_id"]==match_id]
        set_nums = match_df["set"].unique()
        for set_num in set_nums:
            set_df = match_df[match_df["set"]==set_num]
            Evaluation(set_df, f'../Shot_Evaluation/Result/{match_id}/{set_num}')




if __name__ == "__main__":
    File = "../input_data/All_dataset.csv"
    main(File)


/home/derrick/Downloads/from_gdwang-New_Environment/Shot_Evaluation/Utils/Dataprocess.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['type'] = df['type'].map(type)
/home/derrick/Downloads/from_gdwang-New_Environment/Shot_Evaluation/Utils/Dataprocess.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lose_reason'] = df['lose_reason'].map(lose_reason)
/home/derrick/Downloads/from_gdwang-New_Environment/Shot_Evaluation/Utils/Dataprocess.py:36: SettingWithCopyWarning: 
A value is trying to be set

Players: ['Viktor AXELSEN' 'Anthony Sinisuka GINTING']


posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
/home/derrick/Downloads/from_gdwang-New_Environment/Shot_Evaluation/shot_influence/plot_shot_influence.py:466: UserWarning: Ti

Players: ['Anthony Sinisuka GINTING' 'Viktor AXELSEN']


posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and posy should be finite values
posx and pos

In [9]:
import pandas as pd
data = pd.read_csv("../all_dataset.csv")
#data = pd.read_csv("../dataset_all.csv")
data['match_id'] = data['match_id'].astype(int).astype(str)

match_id_mapping = {'1': '23', '3': '28', '5': '30', '6': '31', '7': '32', '13': '49', '2': '25', '4': '29', '8': '36', '9': '43', '10': '44',
                    '11': '45', '19': '55', '14': '50', '15': '51', '30': '72', '36': '79', '44': '97', '17': '53', '26': '64', '12': '48',
                    '20': '56', '23': '60', '28': '69', '41': '88', '18': '54', '16': '52', '27': '66', '33': '75', '37': '82', '24': '61',
                    '39': '86', '21': '57', '34': '76', '29': '71', '35': '78', '38': '85', '22': '58', '32': '74', '25': '63', '31': '73',
                    '43': '94', '42': '89', '40': '87'}

data['match_id'] = data['match_id'].map(match_id_mapping)
data.head()


,rally,ball_round,time,frame_num,score_status,player_score,opponent_score,player,server,type,...,player_move_area,moving_x,moving_y,landing_court_number,ball_distance_x,ball_distance_y,db,set,match_id,rally_id
0,1,1,0:07:39,11496.0,0.0,0.0,0.0,Kento MOMOTA,1.0,發短球,...,11.0,0.012041,0.262869,NaN,0.000000,0.000000,0.0,1,23,0
1,1,2,00:07:43,11582.0,0.0,0.0,0.0,CHOU Tien Chen,3.0,長球,...,18.0,-0.073337,0.594564,NaN,0.089945,0.308945,0.0,1,23,0
2,2,1,0:07:55,11881.0,1.0,1.0,0.0,Kento MOMOTA,1.0,發短球,...,11.0,0.011994,0.275664,NaN,0.000000,0.000000,0.0,1,23,1
3,2,2,0:07:56,11900.0,1.0,1.0,0.0,CHOU Tien Chen,2.0,推撲球,...,10.0,-0.058087,0.142593,NaN,-0.032041,0.210822,0.0,1,23,1
4,2,3,0:07:56,11921.0,1.0,1.0,0.0,Kento MOMOTA,2.0,殺球,...,7.0,0.158490,-0.000140,NaN,0.299495,-0.413447,0.0,1,23,1
